# **DATA INPUT**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np
import itertools
from joblib import Parallel, delayed
import os
import multiprocessing
import json
import random

import sklearn.model_selection as skm
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import ParameterGrid, GridSearchCV
from sklearn.svm import SVR

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
import statsmodels.api as sm

from scipy.interpolate import Akima1DInterpolator
from scipy.interpolate import barycentric_interpolate

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LSTM, Dropout, SimpleRNN, GRU, Input, Concatenate
from tensorflow.keras.optimizers import Adam, Adadelta, RMSprop, Adamax, Adagrad, Nadam
from tensorflow.keras.activations import elu, relu, swish
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
datas = pd.read_csv("aaaaa.csv")

# Index Generator

In [ ]:
random.seed(42)

This was made to ensure the same indices were removed as the test set for all the models.

In [ ]:
def generate_non_overlapping_indices(num_observations, num_series, length_series):
    max_start_index = num_observations - length_series
    available_indices = set(range(max_start_index + 1))  # Create a set of all possible start indices
    chosen_indices = []

    while len(chosen_indices) < num_series:
        start_index = random.choice(list(available_indices))  # Randomly pick from available indices
        if all(start_index + i in available_indices for i in range(length_series)):  # Check all needed indices are available
            chosen_indices.append(start_index)
            # Mark the indices of this block as unavailable
            for i in range(length_series):
                available_indices.discard(start_index + i)

    return sorted(chosen_indices)

def expand_indices(start_indices, sequence_length):
    full_indices = []
    for start in start_indices:
        # Append each index from start to start + sequence_length - 1
        full_indices.extend(range(start, start + sequence_length))
    return full_indices

num_observations = 3964

sequence_length_week = 7
sequence_length_month = 31

length_series_week = 7
length_series_month = 31

In [ ]:
final_indices_week = []
final_indices_month = []

In [ ]:
for i in range(3):
    indices_week = generate_non_overlapping_indices(num_observations, 50, length_series_week)
    indices_month = generate_non_overlapping_indices(num_observations, 10, length_series_month)

    expanded_indices_week = expand_indices(indices_week, sequence_length_week)
    expanded_indices_month = expand_indices(indices_month, length_series_month)

    final_indices_week.append(expanded_indices_week)
    final_indices_month.append(expanded_indices_month)

In [ ]:
expanded_indices_month = final_indices_month[1]

#**Benchmark Models**

In [ ]:
data = datas.copy()
date_indices = data.index[expanded_indices_month]

In [ ]:
data['Date'] = pd.to_datetime(data[['Year', 'Month', 'Day']])

# Set 'Date' as the index
data.set_index('Date', inplace=True)

In [ ]:
data = data[['Concentration']]

In [ ]:
### We create new dataframes to interpolate
data_linear = data.copy()
data_poly2 = data.copy()
data_poly3 = data.copy()
data_poly5 = data.copy()
data_neighbor = data.copy()
data_time = data.copy()
data_spline = data.copy()
data_piecewise = data.copy()
data_akima = data.copy()
data_average = data.copy()
data_median = data.copy()

## Linear Interpolation

In [ ]:
# Step 1: Dataset copy
data_linear_copy = data_linear.copy()

In [ ]:
# Seed for reporducibility
np.random.seed(42)

In [ ]:
# Step 2: Manually introduce NaN values
data_linear_copy.loc[date_indices, 'Concentration'] = np.nan

In [ ]:
# Step 3: Interpolate the deleted NaN values
data_linear_copy['Concentration_interpolated'] = data_linear_copy['Concentration'].interpolate()

In [ ]:
# Step 4: Take only interpolated values and their corresponding dates
interpolated_data_linear = data_linear_copy[pd.isna(data_linear_copy['Concentration'])]

In [ ]:
columns_to_remove = ['Concentration']

# Use drop to remove the specified columns
interpolated_data_linear = interpolated_data_linear.drop(columns=columns_to_remove)

In [ ]:
# Step 5: Merge (via right join) the interpolated dataset with the original dataset using the index
merged_data = pd.merge(data_linear, interpolated_data_linear, how='right', left_index=True, right_index=True)

In [ ]:
# Step 6: Compute the MSE and MAE
mse = mean_squared_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
mae = mean_absolute_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Squared Error (MSE): {mse:.2f}')
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
df_a = merged_data[['Concentration']]
df_b = merged_data[['Concentration_interpolated']]
values_real = df_a.values.ravel()  # Flatten values to 1D
values_inter = df_b.values.ravel()  # Flatten values to 1D
merged_data = merged_data.reset_index(drop=True)
# Create a dataframe combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': values_real, 'Predicted_Concentration': values_inter}, index=merged_data.index)

These are the codes for the 2 graphs that are used for all of the imputations.

In [ ]:
### Variance Graph

plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# To add the perfect prediction line.
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
### Scatter Graph

plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')

for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

## 2nd Degree Polynomial Interpolation

In [ ]:
# Step 1: Dataset copy
data_poly2_copy = data_poly2.copy()

In [ ]:
# Seed for reporducibility
np.random.seed(42)

In [ ]:
# Step 2: Manually introduce NaN values
data_poly2_copy.loc[date_indices, 'Concentration'] = np.nan

In [ ]:
# Step 3: Interpolate the deleted NaN values with a second-degree polynomial
data_poly2_copy['Concentration_interpolated'] = data_poly2_copy['Concentration'].interpolate(method='polynomial', order=2)

In [ ]:
# Step 4: Take only interpolated values and their corresponding dates
interpolated_data_poly2 = data_poly2_copy[pd.isna(data_poly2_copy['Concentration'])]

In [ ]:
columns_to_remove = ['Concentration']

# Use drop to remove the specified columns
interpolated_data_poly2 = interpolated_data_poly2.drop(columns=columns_to_remove)

In [ ]:
# Step 5: Merge (via right join) the interpolated dataset with the original dataset using the index
merged_data = pd.merge(data_poly2, interpolated_data_poly2, how='right', left_index=True, right_index=True)

In [ ]:
# Step 6: Calculate the MSE
mse = mean_squared_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
df_a = merged_data[['Concentration']]
df_b = merged_data[['Concentration_interpolated']]
values_real = df_a.values.ravel()  # Flatten values to 1D
values_inter = df_b.values.ravel()  # Flatten values to 1D
merged_data = merged_data.reset_index(drop=True)
# Create a dataframe combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': values_real, 'Predicted_Concentration': values_inter}, index=merged_data.index)

In [ ]:
### Variance Graph

plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# To add the perfect prediction line.
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
### Scatter Graph

plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')

for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

## 3rd Degree Polynomial Interpolation

In [ ]:
# Step 1: Dataset copy
data_poly3_copy = data_poly3.copy()

In [ ]:
# Seed for reporducibility
np.random.seed(42)

In [ ]:
# Step 2: Manually introduce NaN values
data_poly3_copy.loc[date_indices, 'Concentration'] = np.nan

In [ ]:
# Step 3: Interpolate the deleted NaN values with a third-degree polynomial
data_poly3_copy['Concentration_interpolated'] = data_poly3_copy['Concentration'].interpolate(method='polynomial', order=3)

In [ ]:
# Step 4: Take only interpolated values and their corresponding dates
interpolated_data_poly3 = data_poly3_copy[pd.isna(data_poly3_copy['Concentration'])]

In [ ]:
columns_to_remove = ['Concentration']

# Use drop to remove the specified columns
interpolated_data_poly3 = interpolated_data_poly3.drop(columns=columns_to_remove)

In [ ]:
# Step 5: Merge (via right join) the interpolated dataset with the original dataset using the index
merged_data = pd.merge(data_poly3, interpolated_data_poly3, how='right', left_index=True, right_index=True)

In [ ]:
# Step 6: Calculate the MSE
mse = mean_squared_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
df_a = merged_data[['Concentration']]
df_b = merged_data[['Concentration_interpolated']]
values_real = df_a.values.ravel()  # Flatten values to 1D
values_inter = df_b.values.ravel()  # Flatten values to 1D
merged_data = merged_data.reset_index(drop=True)
# Create a dataframe combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': values_real, 'Predicted_Concentration': values_inter}, index=merged_data.index)

In [ ]:
### Variance Graph

plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# To add the perfect prediction line.
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
### Scatter Graph

plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')

for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

## Nearest Neighbor Interpolation

In [ ]:
# Step 1: Dataset copy
data_neighbor_copy = data_neighbor.copy()

In [ ]:
# Seed for reporducibility
np.random.seed(42)

In [ ]:
# Step 2: Manually introduce NaN values
data_neighbor_copy.loc[date_indices, 'Concentration'] = np.nan

In [ ]:
# Step 3: Interpolate the deleted NaN values with nearest neighbor interpolation
data_neighbor_copy['Concentration_interpolated'] = data_neighbor_copy['Concentration'].interpolate(method='nearest')

In [ ]:
# Step 4: Take only interpolated values and their corresponding dates
interpolated_data_nearest = data_neighbor_copy[pd.isna(data_neighbor_copy['Concentration'])]

In [ ]:
columns_to_remove = ['Concentration']

# Use drop to remove the specified columns
interpolated_data_nearest = interpolated_data_nearest.drop(columns=columns_to_remove)

In [ ]:
# Step 5: Merge (via right join) the interpolated dataset with the original dataset using the index
merged_data = pd.merge(data_neighbor, interpolated_data_nearest, how='right', left_index=True, right_index=True)

In [ ]:
# Step 6: Calculate the MSE
mse = mean_squared_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Absolute Error (MAE): {mae:.2f}')

Mean Squared Error (MSE): 299.97
Mean Absolute Error (MAE): 13.36


In [ ]:
df_a = merged_data[['Concentration']]
df_b = merged_data[['Concentration_interpolated']]
values_real = df_a.values.ravel()  # Flatten values to 1D
values_inter = df_b.values.ravel()  # Flatten values to 1D
merged_data = merged_data.reset_index(drop=True)
# Create a dataframe combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': values_real, 'Predicted_Concentration': values_inter}, index=merged_data.index)

In [ ]:
### Variance Graph

plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# To add the perfect prediction line.
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
v### Scatter Graph

plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')

for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

## Spline Interpolation

In [ ]:
# Step 1: Dataset copy
data_spline_copy = data_spline.copy()

In [ ]:
# Seed for reporducibility
np.random.seed(42)

In [ ]:
# Step 2: Manually introduce NaN values
data_spline_copy.loc[date_indices, 'Concentration'] = np.nan

In [ ]:
# Step 3: Interpolate the deleted NaN values with spline interpolation
data_spline_copy['Concentration_interpolated'] = data_spline_copy['Concentration'].interpolate(method='spline', order=2)

In [ ]:
# Step 4: Take only interpolated values and their corresponding dates
interpolated_data_spline = data_spline_copy[pd.isna(data_spline_copy['Concentration'])]

In [ ]:
columns_to_remove = ['Concentration']

# Use drop to remove the specified columns
interpolated_data_poly2 = interpolated_data_poly2.drop(columns=columns_to_remove)

In [ ]:
# Step 5: Merge (via right join) the interpolated dataset with the original dataset using the index
merged_data = pd.merge(data_spline, interpolated_data_spline, how='right', left_index=True, right_index=True)

In [ ]:
# Step 6: Calculate the MSE
mse = mean_squared_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
df_a = merged_data[['Concentration']]
df_b = merged_data[['Concentration_interpolated']]
values_real = df_a.values.ravel()  # Flatten values to 1D
values_inter = df_b.values.ravel()  # Flatten values to 1D
merged_data = merged_data.reset_index(drop=True)
# Create a dataframe combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': values_real, 'Predicted_Concentration': values_inter}, index=merged_data.index)

In [ ]:
### Variance Graph

plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# To add the perfect prediction line.
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
### Scatter Graph

plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')

for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

## Piecewise Polynomial Interpolation

In [ ]:
# Step 1: Dataset copy
data_piecewise_copy = data_piecewise.copy()

In [ ]:
# Seed for reporducibility
np.random.seed(42)

In [ ]:
# Step 2: Manually introduce NaN values
data_piecewise_copy.loc[date_indices, 'Concentration'] = np.nan

In [ ]:
# Step 3: Interpolate the deleted NaN values with piecewise polynomial
data_piecewise_copy['Concentration_interpolated'] = data_piecewise_copy['Concentration'].interpolate(method='pchip')

In [ ]:
# Step 4: Take only interpolated values and their corresponding dates
interpolated_data_piece = data_piecewise_copy[pd.isna(data_piecewise_copy['Concentration'])]

In [ ]:
columns_to_remove = ['Concentration']

# Use drop to remove the specified columns
interpolated_data_piece = interpolated_data_piece.drop(columns=columns_to_remove)

In [ ]:
# Step 5: Merge (via right join) the interpolated dataset with the original dataset using the index
merged_data = pd.merge(data_piecewise, interpolated_data_piece, how='right', left_index=True, right_index=True)

In [ ]:
# Step 6: Calculate the MSE
mse = mean_squared_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
df_a = merged_data[['Concentration']]
df_b = merged_data[['Concentration_interpolated']]
values_real = df_a.values.ravel()  # Flatten values to 1D
values_inter = df_b.values.ravel()  # Flatten values to 1D
merged_data = merged_data.reset_index(drop=True)
# Create a dataframe combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': values_real, 'Predicted_Concentration': values_inter}, index=merged_data.index)

In [ ]:
### Variance Graph

plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# To add the perfect prediction line.
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
### Scatter Graph

plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')

for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

## Akima Interpolation

In [ ]:
# Step 1: Dataset copy
data_akima_copy = data_akima.copy()

In [ ]:
# Seed for reporducibility
np.random.seed(42)

In [ ]:
# Step 2: Manually introduce NaN values
data_akima_copy.loc[date_indices, 'Concentration'] = np.nan

In [ ]:
nan_mask = pd.isna(data_akima_copy['Concentration'])

# Step 3: Create an Akima interpolator for non-NaN values
akima_interpolator = Akima1DInterpolator(data_akima_copy.index[~nan_mask], data_akima_copy['Concentration'].dropna())

In [ ]:
# Step 4: Interpolate the deleted NaN values with akima interpolation
data_akima_copy.loc[nan_mask, 'Concentration_interpolated'] = akima_interpolator(data_akima_copy.index[nan_mask])

In [ ]:
columns_to_remove = ['Concentration']

# Use drop to remove the specified columns
interpolated_data_akima = interpolated_data_akima.drop(columns=columns_to_remove)

In [ ]:
# Step 4: Take only interpolated values and their corresponding dates
interpolated_data_akima = data_akima_copy[nan_mask]

In [ ]:
# Step 6: Merge (via right join) the interpolated dataset with the original dataset using the index
merged_data = pd.merge(data_akima, interpolated_data_akima, how='right', left_index=True, right_index=True)

In [ ]:
# Step 7: Calculate the MSE
mse = mean_squared_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(merged_data['Concentration'], merged_data['Concentration_interpolated'])
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
df_a = merged_data[['Concentration']]
df_b = merged_data[['Concentration_interpolated']]
values_real = df_a.values.ravel()  # Flatten values to 1D
values_inter = df_b.values.ravel()  # Flatten values to 1D
merged_data = merged_data.reset_index(drop=True)
# Create a dataframe combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': values_real, 'Predicted_Concentration': values_inter}, index=merged_data.index)

In [ ]:
### Variance Graph

plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# To add the perfect prediction line.
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
### Scatter Graph

plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')

for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Support Vector Machines

In [ ]:
data = datas.copy()

In [ ]:
# Define features (X) and target variable (y)
X = data.drop(columns=['Concentration'])
y = data['Concentration']

In [ ]:
# Step 1: Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()  # Separate scaler for the target variable

In [ ]:
# Step 2: Split the data into training and testing sets
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
# Step 3: Fit and transform the scaler on the training features and target
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Step 4: Transform the testing features and target using the fitted scaler
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Initialize the Support Vector Regression model
svm_model = SVR()

# Define a broader hyperparameter grid to search
param_grid = {
    'kernel': ['rbf'],  # Exclude linear kernel
    'C': [7],
    'gamma': [0.003],
    'epsilon': [0.1],
    'shrinking': [True, False],
}

In [ ]:
# Calculate the total number of combinations
total_combinations = len(list(ParameterGrid(param_grid)))
print("Total number of different models being tried:", total_combinations)

In [ ]:
# Initialize GridSearchCV with SVR, hyperparameter grid, and mean squared error scoring
grid_search = GridSearchCV(svm_model, param_grid, scoring='neg_mean_squared_error', cv=10)

In [ ]:
# Fit the model with GridSearchCV on the normalized training set
grid_search.fit(X_train_scaled, y_train_scaled.ravel())

GridSearchCV(cv=10, estimator=SVR(),
             param_grid={'C': [4], 'epsilon': [0.1], 'gamma': [0.006],
                         'kernel': ['rbf'], 'shrinking': [True]},
             scoring='neg_mean_squared_error')

In [ ]:
# Get the best hyperparameters
best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)

In [ ]:
# Get the best model
best_svm_model = grid_search.best_estimator_

In [ ]:
# Test the model on the normalized test set
y_test_pred_scaled = best_svm_model.predict(X_test_scaled)

In [ ]:
# Inverse transform the predictions to the original scale
y_test_pred_scaled = y_test_pred_scaled.reshape(-1, 1)
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)

In [ ]:
# Inverse transform the predictions to the original scale
y_test_pred = scaler_y.inverse_transform(y_test_pred_scaled)

In [ ]:
# MSE and MAE computations
mse = mean_squared_error(y_test, y_test_pred)
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(y_test, y_test_pred)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
predictions_flat = np.squeeze(y_test_pred)
actual_values_flat = y_test.values.ravel()
y_test = y_test.reset_index(drop=True)

# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()



plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Feedforward Neural Networks

In [ ]:
data = datas.copy()

In [ ]:
# Define features (X) and target variable (y)
X = data.drop(columns=['Concentration'])
y = data['Concentration']

In [ ]:
# Step 1: Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()  # Separate scaler for the target variable

In [ ]:
# Step 2: Split the data into training and testing sets
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
# Step 3: Fit and transform the scaler on the training features and target
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Step 4: Transform the testing features and target using the fitted scaler
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Step 5: Define and compile the neural network model
model = Sequential()
model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='elu'))
model.add(Dense(32, activation='elu', kernel_regularizer=l2(0.0001)))  # L2 regularization
model.add(Dense(16, activation='elu', kernel_regularizer=l2(0.0001)))
model.add(Dense(8, activation='elu', kernel_regularizer=l2(0.0001)))
model.add(Dense(4, activation='elu', kernel_regularizer=l2(0.0001)))
model.add(Dense(1, activation='linear'))

# Define the Adamax optimizer
optimizer = Adamax()

model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

In [ ]:
# Step 6: Fit the model on the training data
early_stopping = EarlyStopping(monitor='val_loss', patience=300, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_scaled, epochs=650, batch_size=7,
                    verbose=1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
# Step 7: After making predictions, inverse transform the predictions
predictions_scaled = model.predict(X_test_scaled)
predictions_original_scale = scaler_y.inverse_transform(predictions_scaled)

In [ ]:
# Step 8: MSE and MAE computations
mse = mean_squared_error(y_test, predictions_original_scale)
print(f'Mean Squared Error (MSE): {mse:.2f}')

# Calculate the Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, predictions_original_scale)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
# Step 9: Graph plotting

predictions_flat = np.squeeze(predictions_original_scale)
actual_values_flat = y_test.values.ravel()  # This will flatten to 1D
y_test = y_test.reset_index(drop=True)

# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()




plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Recurrent Neural Networks

In [ ]:
data = datas.copy()

In [ ]:
# Define features (X) and target variable (y)
X = data.drop(columns=['Concentration'])
y = data['Concentration']

In [ ]:
# Step 1: Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()  # Separate scaler for the target variable

In [ ]:
# Step 2: Split the data into training and testing sets
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
# Step 3: Fit and transform the scaler on the training features and target
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Step 4: Transform the testing features and target using the fitted scaler
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Reshape the input data for RNN
X_train_scaled_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [ ]:
# Define the RNN model with L2 regularization
model = Sequential()
model.add(SimpleRNN(16, input_shape=(X_train_scaled_reshaped.shape[1], X_train_scaled_reshaped.shape[2]), return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(SimpleRNN(8, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(SimpleRNN(4, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(SimpleRNN(2, return_sequences=False, activation=elu, kernel_regularizer=l2(0.001)))  # Last layer
model.add(Dense(1, activation='linear'))

# Compile the model
optimizer = Adamax()
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
# Step 5: Fit the model on the training data
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_scaled, epochs=600, batch_size=7,
                	verbose=1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
# Step 6: After making predictions, inverse transform the predictions
predictions_scaled = model.predict(X_test_scaled)
predictions_original_scale = scaler_y.inverse_transform(predictions_scaled)

In [ ]:
# Step 7: MSE and MAE computations
mse = mean_squared_error(y_test, predictions_original_scale)
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(y_test, predictions_original_scale)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
# Step 8: Graph plotting
predictions_flat = np.squeeze(predictions_original_scale)
actual_values_flat = y_test.values.ravel()  # This will flatten to 1D
y_test = y_test.reset_index(drop=True)

# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()


plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Gated Recurrent Networks

In [ ]:
data = datas.copy()

In [ ]:
# Define features (X) and target variable (y)
X = data.drop(columns=['Concentration'])
y = data['Concentration']

In [ ]:
# Step 1: Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()  # Separate scaler for the target variable

In [ ]:
# Step 2: Split the data into training and testing sets
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
# Step 3: Fit and transform the scaler on the training features and target
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Step 4: Transform the testing features and target using the fitted scaler
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Reshape the input data for RNN
X_train_scaled_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [ ]:
# Define the GRU model with L2 regularization
model = Sequential()
model.add(GRU(16, input_shape=(X_train_scaled_reshaped.shape[1], X_train_scaled_reshaped.shape[2]), return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(GRU(8, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(GRU(4, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(GRU(2, return_sequences=False, activation=elu, kernel_regularizer=l2(0.001)))  # Last layer
model.add(Dense(1, activation='linear'))

# Compile the model
optimizer = Adamax()
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
# Step 5: Fit the model on the training data
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_scaled, epochs=600, batch_size=7,
                	verbose=1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
# Step 6: After making predictions, inverse transform the predictions
predictions_scaled = model.predict(X_test_scaled)
predictions_original_scale = scaler_y.inverse_transform(predictions_scaled)

In [ ]:
# Step 7: MSE and MAE computations
mse = mean_squared_error(y_test, predictions_original_scale)
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(y_test, predictions_original_scale)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
# Step 8: Graph plotting
predictions_flat = np.squeeze(predictions_original_scale)
actual_values_flat = y_test.values.ravel()  # This will flatten to 1D
y_test = y_test.reset_index(drop=True)

# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()



plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Long Short-Term Memory

In [ ]:
data = datas.copy()

In [ ]:
# Define features (X) and target variable (y)
X = data.drop(columns=['Concentration'])
y = data['Concentration']

In [ ]:
# Step 1: Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()  # Separate scaler for the target variable

In [ ]:
# Step 2: Split the data into training and testing sets
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
# Step 3: Fit and transform the scaler on the training features and target
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Step 4: Transform the testing features and target using the fitted scaler
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Reshape the input data for LSTM
X_train_scaled_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [ ]:
# Step 5: Define the LSTM model
model = Sequential()
model.add(LSTM(16, input_shape=(X_train_scaled_reshaped.shape[1], X_train_scaled_reshaped.shape[2]), return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(LSTM(8, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(LSTM(4, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))
model.add(LSTM(2, return_sequences=False, activation=elu, kernel_regularizer=l2(0.001)))  # Last layer
model.add(Dense(1, activation='elu'))

# Compile the model
optimizer = Adamax()
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
# Step 6: Fit the model on the training data
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_scaled, epochs=600, batch_size=7,
                    verbose=1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
# Step 7: After making predictions, inverse transform the predictions
predictions_scaled = model.predict(X_test_scaled)
predictions_original_scale = scaler_y.inverse_transform(predictions_scaled)

In [ ]:
# Step 8: MSE and MAE computations
mse = mean_squared_error(y_test, predictions_original_scale)
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(y_test, predictions_original_scale)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
# Step 9: Graph plotting
predictions_flat = np.squeeze(predictions_original_scale)
actual_values_flat = y_test.values.ravel()  # This will flatten to 1D
y_test = y_test.reset_index(drop=True)

# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()


plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Mixed Layer Model

In [ ]:
data = datas.copy()

In [ ]:
# Step 1: Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()  # Separate scaler for the target variable

In [ ]:
# Define features (X) and target variable (y)
X = data.drop(columns=['Concentration'])
y = data['Concentration']

In [ ]:
# Step 2: Split the data into training and testing sets
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
# Step 3: Fit and transform the scaler on the training features and target
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Step 4: Transform the testing features and target using the fitted scaler
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Reshape the input data for RNN
X_train_scaled_reshaped = X_train_scaled.reshape(X_train_scaled.shape[0], X_train_scaled.shape[1], 1)
X_test_scaled_reshaped = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)

In [ ]:
# Step 5: Define the model
model = Sequential()

# Add the first recurrent layer (GRU)
model.add(SimpleRNN(16, input_shape=(X_train_scaled_reshaped.shape[1], X_train_scaled_reshaped.shape[2]), return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))

# Add the second recurrent layer (LSTM)
model.add(SimpleRNN(8, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))

# Add the third recurrent layer (SimpleRNN)
model.add(SimpleRNN(4, return_sequences=True, activation=elu, kernel_regularizer=l2(0.001)))

# Add the fourth recurrent layer (GRU)
model.add(GRU(2, return_sequences=False, activation=elu, kernel_regularizer=l2(0.001)))  # Last layer

# Add the output layer
model.add(Dense(1))

# Compile the model
optimizer = Adamax()
model.compile(optimizer=optimizer, loss='mse')

In [ ]:
# Step 6: Fit the model on the training data
early_stopping = EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train_scaled, epochs=600, batch_size=7,
                    verbose=1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
# Step 7: After making predictions, inverse transform the predictions
predictions_scaled = model.predict(X_test_scaled)
predictions_original_scale = scaler_y.inverse_transform(predictions_scaled)

In [ ]:
# Step 8: MSE and MAE computations
mse = mean_squared_error(y_test, predictions_original_scale)
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(y_test, predictions_original_scale)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
# Step 9: Graph plotting
predictions_flat = np.squeeze(predictions_original_scale)
actual_values_flat = y_test.values.ravel()  # This will flatten to 1D
y_test = y_test.reset_index(drop=True)

# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.savefig("mixed_daily_variance.pdf")
plt.show()


plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Autoencoders

In [ ]:
data = datas.copy()

In [ ]:
# Define features (X) and target variable (y)
X = data.drop(columns=['Concentration'])
y = data[['Concentration']]

In [ ]:
# Step 1: Initialize the MinMaxScaler
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()  # Separate scaler for the target variable

In [ ]:
# Set a seed for reproducibility
np.random.seed(42)

In [ ]:
# Step 2: Split the data into training and testing sets
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
# Step 3: Fit and transform the scaler on the training features and target
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
# Step 4: Transform the testing features and target using the fitted scaler
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Define the autoencoder architecture
n_features = X_train_scaled_reshaped.shape[1]  # This can be adjusted to your specific number of features

# Dynamically create input layers
input_layers = [Input(shape=(1,)) for _ in range(n_features)]

# Concatenate all input layers
concatenated_inputs = Concatenate()(input_layers)

# Encoder layers
encoded = Dense(70, activation='elu', kernel_regularizer=l2(0.0001))(concatenated_inputs)  # First hidden layer
encoded = Dense(35, activation='elu', kernel_regularizer=l2(0.0001))(encoded)
encoded = Dense(21, activation='elu', kernel_regularizer=l2(0.0001))(encoded)      # Second hidden layer
encoded = Dense(14, activation='elu', kernel_regularizer=l2(0.0001))(encoded)      # Third hidden layer
encoded = Dense(7, activation='elu', kernel_regularizer=l2(0.0001))(encoded)      # Fourth hidden layer
encoded = Dense(1, activation='elu', kernel_regularizer=l2(0.0001))(encoded)  # Encoding layer

# Decoder layers
decoded = Dense(7, activation='elu', kernel_regularizer=l2(0.0001))(encoded)      # First hidden layer in decoder
decoded = Dense(14, activation='elu', kernel_regularizer=l2(0.0001))(decoded)      # Second hidden layer in decoder
decoded = Dense(21, activation='elu', kernel_regularizer=l2(0.0001))(decoded)      # Third hidden layer in decoder
decoded = Dense(35, activation='elu', kernel_regularizer=l2(0.0001))(decoded)      # Fourth hidden layer in decoder
decoded = Dense(70, activation='elu', kernel_regularizer=l2(0.0001))(decoded)
decoded = Dense(1, activation='linear')(decoded)

autoencoder = Model(inputs=input_layers, outputs=decoded)

# Step 5: Compile the model
autoencoder.compile(optimizer='adamax', loss='mse', metrics=['mse'])

X_train_list = [X_train_scaled[:, i:i+1] for i in range(n_features)]

early_stopping = EarlyStopping(monitor='val_loss', patience=200, restore_best_weights=True)

history = autoencoder.fit(X_train_list, y_train_scaled[:, 0], epochs=15000, batch_size=7, verbose=1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
# Step 6: After making predictions, inverse transform the predictions
X_test_list = [X_test_scaled[:, i:i+1] for i in range(n_features)]  # n_features should match the number used during training
predictions_scaled = autoencoder.predict(X_test_list)

predictions_original_scale = scaler_y.inverse_transform(predictions_scaled)

In [ ]:
# Step 7: MSE and MAE computations
mse = mean_squared_error(y_test, predictions_original_scale)
print(f'Mean Squared Error (MSE): {mse:.2f}')
mae = mean_absolute_error(y_test, predictions_original_scale)
print(f'Mean Absolute Error (MAE): {mae:.2f}')

In [ ]:
predictions_flat = np.squeeze(predictions_original_scale)
actual_values_flat = y_test.values.ravel()  # This will flatten to 1D
y_test = y_test.reset_index(drop=True)

In [ ]:
# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(14, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()

# Generative Adversarial Networks

In [ ]:
data = datas.copy()

In [ ]:
X = data.drop(columns=['Concentration'])
y = data['Concentration']

In [ ]:
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

In [ ]:
np.random.seed(42)

In [ ]:
X_test = X.loc[expanded_indices_month]
X_train = X.drop(expanded_indices_month)

y_test = y.loc[expanded_indices_month]
y_train = y.drop(expanded_indices_month)

In [ ]:
X_train_scaled = scaler_X.fit_transform(X_train)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1))

In [ ]:
X_test_scaled = scaler_X.transform(X_test)
y_test_scaled = scaler_y.transform(y_test.values.reshape(-1, 1))

In [ ]:
# Generator Model
def build_generator():
    model = Sequential()
    model.add(Dense(210, input_dim=X_train_scaled_reshaped.shape[1], activation='elu', kernel_regularizer=l2(0.0001))) # L2 regularization
    model.add(Dense(105, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(70, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(35, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(14, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(7, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(1, activation='linear'))
    return model

# Discriminator Model
def build_discriminator():
    model = Sequential()
    model.add(Dense(210, input_dim=data.shape[1], activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(105, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(70, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(35, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(14, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(7, activation='elu', kernel_regularizer=l2(0.0001)))
    model.add(Dense(1, activation='linear'))
    return model

generator = build_generator()
discriminator = build_discriminator()

generator.compile(optimizer='Adamax', loss='mse', metrics=['mse'])
discriminator.compile(optimizer='Adamax', loss='mse', metrics=['mse'])

# GAN Model
gan_input = layers.Input(shape=(X_train_scaled_reshaped.shape[1],))
generated_y = generator(gan_input)
gan_output = discriminator(tf.concat([gan_input, generated_y], axis=1))

gan = models.Model(gan_input, gan_output)
gan.compile(optimizer='Adamax', loss='binary_crossentropy')


In [ ]:
def train_gan(generator, discriminator, gan, X_train, y_train, epochs=50000, batch_size=7):
      discriminator_losses = []
      generator_losses = []
      for e in range(epochs):
        # Random batch of examples
        idx = np.random.randint(0, X_train_scaled.shape[0], batch_size)
        real_x = X_train_scaled[idx]
        real_y = y_train_scaled[idx]
        real_samples = np.concatenate([real_x, real_y], axis=1)

        generated_y = generator.predict(real_x)
        generated_samples = np.concatenate([real_x, generated_y], axis=1)

        # Labels for real and fake data
        real_y_disc = np.ones((batch_size, 1))
        fake_y_disc = np.zeros((batch_size, 1))

        # Discriminator training
        discriminator.train_on_batch(real_samples, real_y_disc)
        discriminator.train_on_batch(generated_samples, fake_y_disc)

        d_loss_real = discriminator.train_on_batch(real_samples, real_y_disc)
        d_loss_fake = discriminator.train_on_batch(generated_samples, fake_y_disc)
        discriminator_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Generatoror Training
        misleading_targets = np.ones((batch_size, 1))
        generator_loss = gan.train_on_batch(real_x, misleading_targets)
        gan.train_on_batch(real_x, misleading_targets)
        discriminator_losses.append(discriminator_loss)
        generator_losses.append(generator_loss)
        if e % 250 == 0:
            print(f'Epoch {e + 1}/{epochs}, Loss: {discriminator.evaluate(real_samples, real_y_disc, verbose=0)}')
      return discriminator_losses, generator_losses

discriminator_losses, generator_losses = train_gan(generator, discriminator, gan, X_train_scaled, y_train_scaled)

In [ ]:
generated_y_test = generator.predict(X_test_scaled)
predicted_y_test = scaler_y.inverse_transform(generated_y_test)

In [ ]:
mse = mean_squared_error(y_test, predicted_y_test)
print("Mean Squared Error:", mse)

mae = mean_absolute_error(y_test, predicted_y_test)
print("Mean Absolute Error:", mae)

In [ ]:
lossdiscriminator = []
for i in range(len(discriminator_losses)):
    discriminatorlosses = (discriminator_losses[i][0] + discriminator_losses[i][1]) /2
    lossdiscriminator.append(discriminatorlosses)

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(lossdiscriminator, label='Discriminator Loss')
plt.plot(generator_losses, label='Generator Loss')
plt.title('Training Losses of GAN')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
predictions_flat = np.squeeze(predicted_y_test)
actual_values_flat = y_test.values.ravel()  # This will flatten to 1D
y_test = y_test.reset_index(drop=True)

In [ ]:
# Create a DataFrame combining y_test and predictions_original_scale
results_df = pd.DataFrame({'Actual_Concentration': actual_values_flat, 'Predicted_Concentration': predictions_flat}, index=y_test.index)

# Plotting actual vs predicted
plt.figure(figsize=(10, 6))
plt.scatter(results_df['Actual_Concentration'], results_df['Predicted_Concentration'], color='orange', label='Predicted vs Actual')

# Add diagonal line for perfect predictions
plt.plot(results_df['Actual_Concentration'], results_df['Actual_Concentration'], color='blue', linestyle='--', label='Perfect Prediction')

plt.xlabel('Actual Concentration', fontsize = 16)
plt.ylabel('Predicted Concentration', fontsize = 16)
plt.legend(prop={'size': 12})
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 6))

plt.scatter(results_df.index, results_df['Actual_Concentration'], label='Actual Concentration', color='blue')
plt.scatter(results_df.index, results_df['Predicted_Concentration'], label='Predicted Concentration', color='orange', marker='x')


for i in results_df.index:
    actual = results_df['Actual_Concentration'][i]
    predicted = results_df['Predicted_Concentration'][i]
    line_color = 'black' if actual > predicted else 'red'
    plt.plot([i, i], [actual, predicted], color=line_color, linestyle='--', linewidth=1)

plt.xlabel('Test Sample Index', fontsize = 16)
plt.ylabel('Concentration', fontsize = 16)

legend_elements = [
    Line2D([0], [0], color='blue', lw=0, marker='o', label='Actual Concentration', markersize=10),
    Line2D([0], [0], color='orange', lw=0, marker='x', label='Predicted Concentration', markersize=10),
    Line2D([0], [0], color='black', lw=1, linestyle='--', label='Actual > Predicted'),
    Line2D([0], [0], color='red', lw=1, linestyle='--', label='Actual < Predicted')
]

plt.legend(handles=legend_elements, loc='upper right', prop={'size': 11})
plt.xticks([])
plt.tight_layout()
plt.show()